<h1 align=center><font size = 5>Segmenting and Clustering neighbourhoods in the city of Toronto, Canada</font></h1>
<h1 align=center><font size = 3>Younes EL GARRAI</font></h1>

# Import
First, let's import the relevant libraries for our exercise. As proposed, I chose to use BeautifulSoup library for website scrapping.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge beautifulsoup4 --yes # beautifulsoup install
from bs4 import BeautifulSoup # website scraping library

!conda install -c conda-forge geocoder --yes # geocoder install


print('Libraries imported !')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    sci

## 1. From website scraping to *pandas* dataframe

### from website scrapping...

We use *BeautifulSoup* to scrap the website and create a list with relevant data.

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source , 'html.parser')

data_list = []

table = soup.find('table', class_='wikitable sortable')

table_body = table.find('tbody')

rows = table_body.find_all('tr')

# for loop to create a list from wikitable

for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    #print(cols)
    data_list.append(cols)
    
data_list.remove([])

### ... to *pandas* dataframe

Let's now generate the *pandas* dataframe from the list

In [3]:
wiki_df=pd.DataFrame(data_list,columns=['PostalCode','Borough','Neighborhood'])

wiki_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 2. Data wrangling

As required in the exercise, let's keep rows that have an assigned borough ie. Ignore rows with a borough that is **Not assigned**

In [4]:
toronto0_df = wiki_df[wiki_df.Borough != 'Not assigned']

toronto0_df.reset_index(inplace=True, drop = True)

toronto0_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Not assigned
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


Now we apply a function to concatenate neighborhoods having the same Postal Code.

We use the groupby method, and convert the series to a dataframe, after reseting the index.

In [5]:
toronto1_df = toronto0_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).to_frame().reset_index()

toronto1_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


We now use the *where* method to conditionnaly replace any **Not assigned** Neighborhood by its **Borough**

In [6]:
m = toronto1_df.Neighborhood == 'Not assigned'

toronto1_df.Neighborhood.where(~m, toronto1_df.Borough, inplace = True)

toronto1_df.tail(20)

,PostalCode,Borough,Neighborhood
83,M6R,West Toronto,"Parkdale, Roncesvalles"
84,M6S,West Toronto,"Runnymede, Swansea"
85,M7A,Queen's Park,Queen's Park
86,M7R,Mississauga,Canada Post Gateway Processing Centre
87,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
88,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto"
89,M8W,Etobicoke,"Alderwood, Long Branch"
90,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
91,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
92,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


As you can see, the **Not assigned** value at index 85 has been replaced by *Queen's Park*

Let's check the final dataframe size :

In [7]:
toronto1_df.shape

(103, 3)

## 3. Neighborhoods Latitude and Longitude

We use the *geocoder* package as specified in the assignment :

In [8]:
import geocoder # import geocoder

In [9]:
postal_code = 'M1G'

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):

#g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))

g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
  
lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

print(lat_lng_coords)

[43.768369121000035, -79.21758999999997]


Unfortunatly *geocoder.google* doesn't seem to be working in retrieving the latitude and longitude of our postal codes... As proposed in the discussion section of the assignment, I switched to *geocoder.arcgis*... which seems to be working.

In the section below, we loop on the postal codes, to add and complete Latitude and Longitude columns.

In [10]:
postal_code_col = toronto1_df['PostalCode']

n = toronto1_df.shape[0]

for postal_code , index in zip(postal_code_col , range(n)) :
    
    lat_lng_coords = []

    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
  
    lat_lng_coords = g.latlng

    toronto1_df.loc[ index , 'Latitude'] = lat_lng_coords[0]
    
    toronto1_df.loc[ index , 'Longitude'] = lat_lng_coords[1]

toronto1_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


## 4. Exploring neighborhoods in Toronto

In this section, we first visualize the data we collected using *folium*.

In [11]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


After, getting the map location, we generate it, based on *toronto1_df*.

In [92]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto1_df['Latitude'], toronto1_df['Longitude'], toronto1_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Foursquare API to explore the neighborhoods and segment them

In the Foursquare lab, we learned to generate a json file base on a specific GET request URL.

We learned to clean the json and structure it into a *pandas* dataframe.

Let's do the same for our brand new dataframe.

#### Foursquare API credentials

In [13]:
CLIENT_ID = 'OCRRO3JAEHFG4FAAK5GMADMYD2ZIGZYAEQFST1UNRNMBA2DS' # your Foursquare ID
CLIENT_SECRET = 'D4WBCUPRH3G2SLV4YR00OVU45BJID5141YBOSSIO5K3VX4BV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

#### getNearbyVenues function

To generate data about popular venues in our neighborhoods, we use the *getNearbyVenues* function from the previous lab.

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now we run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [15]:
toronto_venues = getNearbyVenues(names=toronto1_df['Neighborhood'],
                                   latitudes=toronto1_df['Latitude'],
                                   longitudes=toronto1_df['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [16]:
print(toronto_venues.shape)
toronto_venues.head()

(2454, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.811525,-79.195517,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
1,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
2,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.765815,-79.175193,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping
4,"Guildwood, Morningside, West Hill",43.765815,-79.175193,Heron Park Community Centre,43.768867,-79.176958,Gym / Fitness Center


Let's check how many venues were returned for each neighborhood

In [17]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,11,11,11,11,11,11
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",15,15,15,15,15,15
"Alderwood, Long Branch",6,6,6,6,6,6
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
Berczy Park,63,63,63,63,63,63
"Birch Cliff, Cliffside West",6,6,6,6,6,6


How many unique categories is there ?

In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 262 uniques categories.


## 5. Neighborhood Analysis and Clustering

As proposed in the lab, let's modify our dataset, using One Hot Encoding :

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# move neighborhood column at the first column

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

cols = list(toronto_onehot) # list of columns

# move the column to head of list using index, pop and insert

cols.insert(0, cols.pop(cols.index('Neighborhood')))

toronto_onehot = toronto_onehot.loc[:, cols]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [20]:
toronto_onehot.shape

(2454, 262)

In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.00,0.030000,0.000000,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.020000,0.000000,0.030000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.02000

In [22]:
toronto_grouped.shape

(97, 262)

#### Let's now print each neighborhood along with the **top 3** most common venues :

In [23]:
num_top_venues = 3

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.06
2   Steakhouse  0.04


----Agincourt----
                venue  freq
0       Shopping Mall  0.18
1              Bakery  0.09
2  Chinese Restaurant  0.09


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0     Sushi Restaurant   0.5
1             Pharmacy   0.5
2  Monument / Landmark   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
           venue  freq
0  Grocery Store  0.13
1    Pizza Place  0.07
2           Park  0.07


----Alderwood, Long Branch----
               venue  freq
0     Sandwich Place  0.17
1       Dance Studio  0.17
2  Convenience Store  0.17


----Bayview Village----
                        venue  freq
0  Construction & Landscaping  0.25
1                     Dog Run  0.25
2                       Trail  0.25


----Bedford Park, Lawrence Manor East----
 

As in the lab, let's create the new dataframe and display the top 10 venues for each neighborhood

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [71]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Hotel,Asian Restaurant,Salad Place,Restaurant,Bar,Japanese Restaurant,Burger Joint
1,Agincourt,Shopping Mall,Pool,Chinese Restaurant,Park,Vietnamese Restaurant,Sushi Restaurant,Discount Store,Skating Rink,Bakery,Supermarket
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Sushi Restaurant,Pharmacy,Yoga Studio,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Japanese Restaurant,Park,Coffee Shop,Sandwich Place,Caribbean Restaurant,Fast Food Restaurant,Liquor Store,Fried Chicken Joint
4,"Alderwood, Long Branch",Convenience Store,Dance Studio,Gym,Athletics & Sports,Pub,Sandwich Place,Fast Food Restaurant,Field,Farmers Market,Fish & Chips Shop
5,Bayview Village,Construction & Landscaping,Trail,Dog Run,Park,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Eastern European Restaurant
6,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Comfort Food Restaurant,Thai Restaurant,Butcher,Café,Liquor Store,Sandwich Place,Sports Club,Restaurant
7,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Steakhouse,Bakery,Lounge,Café,Seafood Restaurant,Breakfast Spot,Cheese Shop
8,"Birch Cliff, Cliffside West",General Entertainment,Park,College Stadium,Skating Rink,Gym Pool,Gym,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
9,"Bloordale Gardens, Eringate, Markland Wood, Ol...",Electronics Store,Carpet Store,Fish & Chips Shop,Shopping Mall,College Rec Center,Grocery Store,Falafel Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space


### Clustering...

We now run *k*-means to cluster the neighborhoods into 5 clusters.

In [72]:
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 2, 1, 1, 2, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [73]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_venues_sorted

toronto_merged = neighborhoods_venues_sorted

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood

toronto_merged = toronto_merged.join(toronto1_df.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
0,1,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Hotel,Asian Restaurant,Salad Place,Restaurant,Bar,Japanese Restaurant,Burger Joint,M5H,Downtown Toronto,43.649700,-79.382582
1,1,Agincourt,Shopping Mall,Pool,Chinese Restaurant,Park,Vietnamese Restaurant,Sushi Restaurant,Discount Store,Skating Rink,Bakery,Supermarket,M1S,Scarborough,43.793940,-79.267976
2,1,"Agincourt North, L'Amoreaux East, Milliken, St...",Sushi Restaurant,Pharmacy,Yoga Studio,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,M1V,Scarborough,43.817685,-79.280187
3,1,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Japanese Restaurant,Park,Coffee Shop,Sandwich Place,Caribbean Restaurant,Fast Food Restaurant,Liquor Store,Fried Chicken Joint,M9V,Etobicoke,43.743145,-79.584664
4,1,"Alderwood, Long Branch",Convenience Store,Dance Studio,Gym,Athletics & Sports,Pub,Sandwich Place,Fast Food Restaurant,Field,Farmers Market,Fish & Chips Shop,M8W,Etobicoke,43.601131,-79.538785


### ... Visualisation

In [91]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

## 6. Examine Clusters

Let's know examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

### Cluster 1 - Sports & restaurants

In [80]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
17,Cedarbrae,Playground,Trail,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Farm,Donut Shop,M1H,Scarborough,43.769688,-79.239440
65,"Moore Park, Summerhill East",Convenience Store,Gym,Playground,Tennis Court,Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,M4T,Central Toronto,43.690685,-79.382946


### Cluster 2 - Lively restaurants

In [90]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]].head(6)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Hotel,Asian Restaurant,Salad Place,Restaurant,Bar,Japanese Restaurant,Burger Joint,M5H,Downtown Toronto,43.649700,-79.382582
1,Agincourt,Shopping Mall,Pool,Chinese Restaurant,Park,Vietnamese Restaurant,Sushi Restaurant,Discount Store,Skating Rink,Bakery,Supermarket,M1S,Scarborough,43.793940,-79.267976
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Sushi Restaurant,Pharmacy,Yoga Studio,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,M1V,Scarborough,43.817685,-79.280187
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Japanese Restaurant,Park,Coffee Shop,Sandwich Place,Caribbean Restaurant,Fast Food Restaurant,Liquor Store,Fried Chicken Joint,M9V,Etobicoke,43.743145,-79.584664
4,"Alderwood, Long Branch",Convenience Store,Dance Studio,Gym,Athletics & Sports,Pub,Sandwich Place,Fast Food Restaurant,Field,Farmers Market,Fish & Chips Shop,M8W,Etobicoke,43.601131,-79.538785
6,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Comfort Food Restaurant,Thai Restaurant,Butcher,Café,Liquor Store,Sandwich Place,Sports Club,Restaurant,M5M,North York,43.735460,-79.419148


### Cluster 3 - Outdoor & sport

In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]].head(5)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
5,Bayview Village,Construction & Landscaping,Trail,Dog Run,Park,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Eastern European Restaurant,M2K,North York,43.781015,-79.380529
8,"Birch Cliff, Cliffside West",General Entertainment,Park,College Stadium,Skating Rink,Gym Pool,Gym,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,M1N,Scarborough,43.696770,-79.259967
12,"CFB Toronto, Downsview East",Food Court,Coffee Shop,Park,Airport,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,M3K,North York,43.738685,-79.467320
15,Caledonia-Fairbanks,Park,Market,Mexican Restaurant,Bakery,Fast Food Restaurant,Gym,Sporting Goods Shop,Women's Store,Beer Store,Fish Market,M6E,York,43.688640,-79.451018
27,Davisville North,Food & Drink Shop,Breakfast Spot,Clothing Store,Gym,Park,Hotel,Fish Market,Flea Market,Fish & Chips Shop,Field,M4P,Central Toronto,43.712755,-79.388514


### Cluster 4 - Services & industrial

In [87]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
34,Downsview Central,Home Service,Business Service,Eastern European Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,M3M,North York,43.733690,-79.496740
74,"Rouge, Malvern",Home Service,Eastern European Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,M1B,Scarborough,43.811525,-79.195517
